In [3]:
import os
import requests
from datetime import datetime
import re
import json  # Para exportar el resumen
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import numpy as np


C:\Users\FERNANDO\AppData\Local\Temp\ipykernel_3820\4042661585.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# VENTA DE ENERGÍA ELECTRICA POR REGION #
---
Tienen como caractrística ser desde RD13093DM hasta RD13117DM, siendo el último de estos la serie de consumo total(NACIONAL)


In [4]:
codigo = 'RD13104DM'
url = f'https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{codigo}/json/2000-01/2024-12' # las fechas de inicio y final tmbn pueden ser delimitadas
response = requests.get(url) # la respuesta viene a ser el output de la consulta de la dirección dada por el API
data=response.json() # lo almacenamos en un elemenot llamado data

In [5]:
#print(data) # como se observa en la data que es en si un diccionario se encuentran las filas de lo que vendria a componer una tabla de registros

#convertimos a Data Frame
series=data['periods']
name=data['config']['series'][0]['name']
partes = name.split(' - ')
partes_1=partes[1].split(' (')
nombre_extraido = partes_1[0]

df = pd.DataFrame(series)
df.columns = ['Fecha', 'Consumo_kWh']
#print(df)
print(name)
print(nombre_extraido) 
# EN ESTA CELDA SE MUESTRA LO QUE SE HARA EN EL BUCLE GENERAL

Venta de energía eléctrica por departamento - La Libertad (gwh)
La Libertad


---
El procedimiento parece simple, ahora tenemos que reptir esto para todos todas las series del bcrp que se encuentren en el rango mencionado
---

---
# CODIGOS #

In [6]:
# vamos del 093 al 117

indices=np.linspace(93,117,(118-93))
indices =indices.tolist() # convertimos los indices en formato array a formato lista para facilitar la extracción
codigos=[]
for elem in indices:
    #print(elem) # esto solo era para darle un check
    if elem<100:
        elem_1='0'+str(int(elem))
    else:
        elem_1=str(int(elem))
    #print(elem_1)
    cod_gen='RD13'+ str(elem_1) +'DM'
    #print(cod_gen)
    codigos+=[cod_gen]

#print(indices)
print(codigos) # hasta aca se tienen los cosigos a scrapear



['RD13093DM', 'RD13094DM', 'RD13095DM', 'RD13096DM', 'RD13097DM', 'RD13098DM', 'RD13099DM', 'RD13100DM', 'RD13101DM', 'RD13102DM', 'RD13103DM', 'RD13104DM', 'RD13105DM', 'RD13106DM', 'RD13107DM', 'RD13108DM', 'RD13109DM', 'RD13110DM', 'RD13111DM', 'RD13112DM', 'RD13113DM', 'RD13114DM', 'RD13115DM', 'RD13116DM', 'RD13117DM']


In [7]:
inicio='2007-01'
fin='2018-03'
fechas = pd.date_range(start=inicio, end=fin, freq='MS') # creamos las fechas para que tengan solo mes y año
fechas_lista = fechas.strftime('%Y-%m').tolist() # ACA ESTAN LAS FECHAS

dep_nom=[]
valores=[]
for codigo in codigos:
    url = f'https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{codigo}/json/{inicio}/{fin}' 
    response = requests.get(url) # la respuesta viene a ser el output de la consulta de la dirección dada por el API
    data_1=response.json() # lo almacenamos en un elemento llamado data
    df = pd.DataFrame(series)
    df.columns = ['Fecha', 'Consumo_kWh']
    #PRIMERO SE OBTIENEN LOS NOMBRES, que será tmbn el nombre de la columna
    nombre_dep=data_1['config']['series'][0]['name']
    partes = nombre_dep.split(' - ')
    partes_1=partes[1].split(' (')
    nombre_extraido = partes_1[0] # TENER EN MENTE QUE NOMBRE EXTRAIDO SERA EL NOMBRE DE LA COLUMNA
    dep_nom+=[nombre_extraido] # a la lista de nombres que tendremos tmbn como un output
    #SEGUNDO SE OBTIENEN LOS VALORES REGISTRADOS
    series=data_1['periods']
    registros=[elem['values'][0] for elem in series]
    valores+=[registros] # se llena 1 elemento nuevo por cada region

print(dep_nom)
print(len(valores)) # son 25 porque al final contiene los valores totales en formato mensual
print(fechas_lista)


['Amazonas', 'Ancash', 'Apurímac', 'Arequipa', 'Ayacucho', 'Cajamarca', 'Cusco', 'Huancavelica', 'Huánuco', 'Ica', 'Junín', 'La Libertad', 'Lambayeque', 'Lima', 'Loreto', 'Madre de Dios', 'Moquegua', 'Pasco', 'Piura', 'Puno', 'San Martín', 'Tacna', 'Tumbes', 'Ucayali', 'Total']
25
['2007-01', '2007-02', '2007-03', '2007-04', '2007-05', '2007-06', '2007-07', '2007-08', '2007-09', '2007-10', '2007-11', '2007-12', '2008-01', '2008-02', '2008-03', '2008-04', '2008-05', '2008-06', '2008-07', '2008-08', '2008-09', '2008-10', '2008-11', '2008-12', '2009-01', '2009-02', '2009-03', '2009-04', '2009-05', '2009-06', '2009-07', '2009-08', '2009-09', '2009-10', '2009-11', '2009-12', '2010-01', '2010-02', '2010-03', '2010-04', '2010-05', '2010-06', '2010-07', '2010-08', '2010-09', '2010-10', '2010-11', '2010-12', '2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06', '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12', '2012-01', '2012-02', '2012-03', '2012-04', '2012-05', '2

In [8]:
columnas = ['Fecha']+dep_nom
#print(len(columnas))
datos=[fechas_lista]+valores
#print(len(datos))

dic_data={}

for i in range(len(datos)): # se arma el diccionario con la data correspondiente a cada columna
    dic_data[columnas[i]]=datos[i]

#print(len(dic_data))

df=pd.DataFrame(dic_data,columns=columnas)
df

,Fecha,Amazonas,Ancash,Apurímac,Arequipa,Ayacucho,Cajamarca,Cusco,Huancavelica,Huánuco,...,Madre de Dios,Moquegua,Pasco,Piura,Puno,San Martín,Tacna,Tumbes,Ucayali,Total
0,2007-01,4.380031,114.858998,4.584659,108.679171,5.921257,40.234119,58.141735,15.123833,8.105586,...,1.814011,133.533319,36.680377,61.086634,22.634528,10.66214,14.010828,9.383187,12.630017,1970.617892
1,2007-02,3.975474,115.442415,4.278411,134.188708,5.367246,37.440528,51.492631,14.157917,7.059248,...,1.76387,113.558773,33.611232,57.053874,20.391357,9.757983,12.593875,9.289076,11.438785,1930.832805
2,2007-03,4.373794,126.527411,4.668321,168.153166,6.099764,42.398638,57.830396,15.654929,8.05191,...,1.674467,129.124834,39.400578,61.542935,21.859417,10.781585,13.999183,10.142338,12.926345,2054.990148
3,2007-04,4.483883,120.953663,4.918931,160.131499,6.033332,41.983135,57.416925,15.624352,7.295965,...,1.928966,135.505979,38.422699,57.234431,22.203006,10.282406,13.372733,9.951977,12.378917,2038.3247
4,2007-05,4.929053,133.048675,5.259515,172.972359,6.27219,40.328593,59.839718,16.113053,7.921184,...,1.705464,138.742101,40.492439,58.758149,22.352069,10.742367,13.458763,10.542051,12.902096,2049.357853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2017-11,3.879953,151.965508,125.390934,412.293584,12.937512,84.094881,167.777699,14.729093,12.430899,...,8.012318,8.67713299999999,58.540035,96.7169294,47.313817,29.8825166,23.070183,16.8820879,24.902831,3670.90729593
131,2017-12,4.610685,157.545065,130.524269,422.492666,12.978579,86.2952819,188.334138,14.774265,12.937574,...,7.682342,9.309747,59.785809,107.5067821,47.058187,30.5932268,24.614872,18.0735585,25.691489,3757.56888003
132,2018-01,4.770503,141.51512,126.6804891,412.6086835,12.759586,86.969426,190.2369943,13.6750257,12.16549,...,7.57068,65.734781,54.6521765,116.6091233,49.9817489,26.6939374,24.3672486,17.998078,24.722539,3774.3142604
133,2018-02,4.401297,132.371681,111.9660692,379.4628053,11.278853,80.1566273,170.8925277,12.4539547,11.510201,...,7.582677,62.087221,47.7742518,117.9471449,47.0136747,25.326772,22.800147,16.8856993,22.866497,3591.4604712
